In [ ]:
import pandas as pd
import openpyxl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf
import math
import os
from dateutil.relativedelta import relativedelta
from sklearn.cluster import KMeans
import xgboost
from xgboost import XGBClassifier
from xgboost import plot_tree
# Logistic Regresison 
import statsmodels
from  statsmodels.discrete.discrete_model import Logit
import sklearn
from sklearn.linear_model import LogisticRegression

In [ ]:
product_sale = pd.read_csv(r'C:\Users\Admin\RBAC\Round3_RBAC\ProductSales.csv')
voucher_map = pd.read_csv(r'C:\Users\Admin\RBAC\Round3_RBAC\VoucherMapping.csv')
product_map = pd.read_csv(r'C:\Users\Admin\RBAC\Round3_RBAC\ProductMapping.csv')

In [ ]:
# Assuming your DataFrame is named product_sale
product_sale['TransactionDate'] = pd.to_datetime(product_sale['TransactionDate'])  # Convert TransactionDate to datetime if it's not already
# Extract quarter and year and combine them into a single column
product_sale['Quarter_Year'] = product_sale['TransactionDate'].dt.to_period('Q')

In [ ]:
ori_date = product_sale[['Quarter_Year', 'CustomerID']].drop_duplicates(subset=['Quarter_Year','CustomerID'])

In [ ]:
ori_date['Active_NextMonth?'] = 1
ori_date

In [ ]:
next_date_tab = ori_date.copy().drop(columns=['Active_NextMonth?']).rename(columns={'Quarter_Year':'Current_Quarter_Year'})
next_date_tab['Next_QuarterYear'] = next_date_tab['Current_Quarter_Year'].apply(lambda x: x+1)
next_date_tab

In [ ]:
merged = next_date_tab.merge(ori_date, left_on = ['Next_QuarterYear','CustomerID'], right_on =['Quarter_Year','CustomerID'], how='left')

In [ ]:
merged = merged.fillna(0).drop(columns=['Quarter_Year'])

In [ ]:
len(merged['CustomerID'].unique())

In [ ]:
merged

# 1 | Second approach

In [ ]:
num_quarter = 5
for i in range(num_quarter):
    print(i)

In [ ]:
def get_churn_customer(df):
    distinct_quarter = df['Quarter_Year'].unique()
    num_quarter = len(distinct_quarter)
    df['customer_type'] = 0

    for i_cur_quarter in range(num_quarter):

        cur_quarter = distinct_quarter[i_cur_quarter]
        
        

        customer_cur_list = df[df['Quarter_Year'] == cur_quarter]['CustomerID'].unique()
        

        if i_cur_quarter == 0:
            next_quarter = distinct_quarter[i_cur_quarter + 1]
            customer_next_list = df[df['Quarter_Year'] == next_quarter]['CustomerID'].unique()
            
            loyal_customer = [customer for customer in customer_cur_list if customer in customer_next_list]
            labser_customer = [customer for customer in customer_cur_list if customer not in loyal_customer]
            new_customer = ['0']
        elif i_cur_quarter == num_quarter - 1:
            last_quarter = distinct_quarter[i_cur_quarter - 1]
            customer_last_list = df[df['Quarter_Year'] == last_quarter]['CustomerID'].unique()
            
            new_customer = [customer for customer in customer_cur_list if customer not in customer_last_list]
            loyal_customer = ['0']
            labser_customer = ['0']
        else:
            next_quarter = distinct_quarter[i_cur_quarter + 1]
            customer_next_list = df[df['Quarter_Year'] == next_quarter]['CustomerID'].unique()
            
            last_quarter = distinct_quarter[i_cur_quarter - 1]
            customer_last_list = df[df['Quarter_Year'] == last_quarter]['CustomerID'].unique()

            new_customer = [customer for customer in customer_cur_list if customer not in customer_last_list]
            not_new_customer = [customer for customer in customer_cur_list if customer not in new_customer]

            loyal_customer = [customer for customer in not_new_customer if customer in customer_next_list]
            labser_customer = [customer for customer in not_new_customer if customer not in loyal_customer]
            

        df['customer_type'][(df['CustomerID'].isin(new_customer))&(df['Quarter_Year'] == cur_quarter)] = 'new'
        df['customer_type'][(df['CustomerID'].isin(loyal_customer))&(df['Quarter_Year'] == cur_quarter)] = 'loyal'
        df['customer_type'][(df['CustomerID'].isin(labser_customer))&(df['Quarter_Year'] == cur_quarter)] = 'labser'

    return df
    

In [ ]:
# Assuming your DataFrame is named product_sale
product_sale['TransactionDate'] = pd.to_datetime(product_sale['TransactionDate'])  # Convert TransactionDate to datetime if it's not already
# Extract quarter and year and combine them into a single column
product_sale['Quarter_Year'] = product_sale['TransactionDate'].dt.to_period('Q')
quarter_customer = product_sale[['Quarter_Year', 'CustomerID']].drop_duplicates(subset=['Quarter_Year','CustomerID'])

In [64]:
distinct_quarter = quarter_customer['Quarter_Year'].unique()
distinct_quarter

<PeriodArray>
['2021Q1', '2021Q2', '2021Q3', '2021Q4', '2022Q1', '2022Q2', '2022Q3',
 '2022Q4']
Length: 8, dtype: period[Q-DEC]

In [59]:
type_customer_over_quarter = get_churn_customer(quarter_customer)

C:\Users\Admin\AppData\Local\Temp\ipykernel_12016\1077953920.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['customer_type'][(df['CustomerID'].isin(new_customer))&(df['Quarter_Year'] == cur_quarter)] = 'new'
C:\Users\Admin\AppData\Local\Temp\ipykernel_12016\1077953920.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['customer_type'][(df['CustomerID'].isin(loyal_customer))&(df['Quarter_Year'] == cur_quarter)] = 'loyal'
C:\Users\Admin\AppData\Local\Temp\ipykernel_12016\1077953920.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'lo

In [60]:
type_customer_over_quarter

,Quarter_Year,CustomerID,customer_type
0,2021Q1,6402,loyal
6,2021Q1,884,labser
11,2021Q1,16274,labser
17,2021Q1,28546,loyal
24,2021Q1,14373,loyal
...,...,...,...
762030,2022Q4,5400,0
762042,2022Q4,19065,new
762048,2022Q4,6687,new
762053,2022Q4,2733,0


In [61]:
# type_customer_over_quarter.to_csv(r'type_customer_over_quarter.csv')